In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Deep Learning Libraries
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def accuracy_loss_plot(history):
    plt.Figure()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='lower right')
    plt.show()
    plt.Figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='lower right')
    plt.show()

In [5]:
data = np.load('/content/drive/MyDrive/mnist/fashion_mnist_dataset_train.npy', allow_pickle=True).item()

In [6]:
s = pd.Series(data['target'])
target = s.replace([1, 2, 3, 4, 5], [0, 1, 2, 3, 4])
target = target.to_numpy()
target

array([4., 1., 0., ..., 2., 0., 3.])

In [7]:
from sklearn.model_selection import train_test_split
# Splitting the data into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(data['features'], target, test_size=0.2, random_state=275)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=275)

In [8]:
print(X_val.shape, '\n')

(6000, 28, 28) 



In [9]:
#pad the images to achieve 32 x 32
X_train = tf.pad(X_train, [[0, 0], [2,2], [2,2]])
X_val = tf.pad(X_val, [[0, 0], [2,2], [2,2]])
X_test = tf.pad(X_test, [[0, 0], [2,2], [2,2]])
#expand and repeat to create 3 channels
X_train = tf.expand_dims(X_train, axis=3, name=None)
X_val = tf.expand_dims(X_val, axis=3, name=None)
X_test = tf.expand_dims(X_test, axis=3, name=None)


In [10]:
print(X_train.shape, '\n')
print(y_train.shape, '\n')
print(X_val.shape, '\n')
print(X_test.shape, '\n')

(48000, 32, 32, 1) 

(48000,) 

(6000, 32, 32, 1) 

(6000, 32, 32, 1) 



In [11]:
print(tf.__version__)

2.4.1


In [12]:
X_train = tf.repeat(X_train, 3, axis=3)
X_val = tf.repeat(X_val, 3, axis=3)
X_test = tf.repeat(X_test, 3, axis=3)

In [13]:
print(X_train.shape, '\n')
print(y_train.shape, '\n')
print(X_val.shape, '\n')
print(X_test.shape, '\n')

(48000, 32, 32, 3) 

(48000,) 

(6000, 32, 32, 3) 

(6000, 32, 32, 3) 



In [14]:
base_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model.layers:
    layer.trainable = False

234700800/234698864 [==============================] - 5s 0us/step


In [15]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(1000, activation='relu')(x)
predictions = layers.Dense(5, activation = 'softmax')(x)

In [16]:
head_model = Model(inputs = base_model.input, outputs = predictions)
head_model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [ ]:
with tf.device('/device:GPU:0'):
  history = head_model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_val, y_val))

Epoch 1/100
375/375 [==============================] - 84s 121ms/step - loss: 1.1084 - accuracy: 0.5446 - val_loss: 0.8643 - val_accuracy: 0.6163
Epoch 2/100
375/375 [==============================] - 41s 109ms/step - loss: 0.7948 - accuracy: 0.6791 - val_loss: 0.7556 - val_accuracy: 0.6943
Epoch 3/100
375/375 [==============================] - 41s 109ms/step - loss: 0.7501 - accuracy: 0.6960 - val_loss: 0.7400 - val_accuracy: 0.6965
Epoch 4/100
375/375 [==============================] - 41s 109ms/step - loss: 0.7239 - accuracy: 0.7030 - val_loss: 0.7172 - val_accuracy: 0.6980
Epoch 5/100
375/375 [==============================] - 41s 109ms/step - loss: 0.6973 - accuracy: 0.7125 - val_loss: 0.6940 - val_accuracy: 0.7130
Epoch 6/100
375/375 [==============================] - 41s 109ms/step - loss: 0.6861 - accuracy: 0.7173 - val_loss: 0.7055 - val_accuracy: 0.6875
Epoch 7/100
375/375 [==============================] - 41s 109ms/step - loss: 0.6706 - accuracy: 0.7202 - val_loss: 0.6538 -

In [ ]:
head_model.evaluate(X_test, y_test)